# Task 3 - Generate a query to capture data quality issues against the new structured relational data model

### In this module, I have provided some data quality analysis I performed in the Data Exploration and Cleaning file. Along with that, I performed extended analysis on the new relational model that will need to be addressed.

In [2]:
# Load libraries
import pandas as pd

In [3]:
# load new datasets
users = pd.read_csv("/Users/rahulnair/Desktop/Fetch/Fetch-Analytics-Engineer-Take-Home/Data/Updated Datasets/users_updated.csv")
receipts = pd.read_csv("/Users/rahulnair/Desktop/Fetch/Fetch-Analytics-Engineer-Take-Home/Data/Updated Datasets/receipts_updated.csv")
brands = pd.read_csv("/Users/rahulnair/Desktop/Fetch/Fetch-Analytics-Engineer-Take-Home/Data/Updated Datasets/brands_updated.csv")
receipts_items_list = pd.read_csv("/Users/rahulnair/Desktop/Fetch/Fetch-Analytics-Engineer-Take-Home/Data/Updated Datasets/receipts_items_list.csv")

### Here are the quality issues that I found in the original datasets. Some of them have been fixed while the rest requires conversation with a SME before I fill it up.
- Brands
    - I noticed that the columns 'catergory' and 'categoryCode' hold similar values. Similarly, 'name' and 'brandCode' hold similar values. Since these are codes that may need to used for tracking purpsoses, we might need to keep it. Good thing is we can use 'name' and 'catergory' for filling up 'brandCode' and 'categoryCode' respectively. 
    - As for the 'category' column, we can fill it with a keyword 'Unknown' for now. We can swap with correct values later. The idea is to not have any misleading values.
    - 'topBrand' is a flag which denotes if that product is a top brand or not. As much as it's easy to fill empty cells with 0, it could prove to be an incorrect info. It would be highly recommended to talk to a SME before filling this column.

- Receipts
    - Dates were in the Epoch format. I fixed that by writing up a function.
    - Columns 'bonusPointsEarnedReason', 'bonusPointsEarned', 'purchasedItemCount' are null because the 'totalSpent' is empty and their 'rewardsReceiptStatus' is either PENDING, FLAGGED or SUBMITTED. We can fill these rows without any issues with '0.0' or 'unknown' (depending on the data type of the field).
    - 'finishedDate' is null because the status of that record is not FINISHED. 'pointsAwardedDate' is null because no points were awarded. I kept it as it is. Not a data quality issues. But just wanted to highlight it.
    - 'pointsEarned' was also filled with 0.0 without any issues.

- Receipts Items List: This one has a lot of null which might hinder analyses. It definitely requires a conversation with a SME before it's filled.

- Users
    - I kept 'lastLogin' NaN. But for columns such as 'signUpSource' and 'state'. We can fill 'unkown'. Once we can get info from am SME, we refill it with correct values.

In [18]:
'''
Following line checks whether the ids in users are unique or not. Turns out, they are not ans this could be fatal for any future analysis. 
For other datasets, this is not an issue.
'''
print(users['id'].is_unique) # Multiple records for same user id
print(brands['id'].is_unique) # No redundancy in id
print(receipts['id'].is_unique) # No redundancy in id

False
True
True


In [23]:
'''
Now let's see if there are lot of null values in the fields that will be used while joining 2 datasets.
'''
df = pd.merge(brands, receipts_items_list, on = 'brandCode', how = 'left')
df

,barcode_x,category,categoryCode,name,topBrand,brandCode,id,cpg_type,cpg_id,barcode_y,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,511111019862,Baking,BAKING,test brand @1612366101024,0.0,TEST BRAND @1612366101024,601ac115be37ce2ead437551,Cogs,601ac114be37ce2ead437550,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,Cogs,5332f5fbe4b03c9a25efd0ba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,Cogs,601ac142be37ce2ead437559,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,Cogs,601ac142be37ce2ead437559,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,Cogs,5332fa12e4b03c9a25efd1e7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153788,511111116752,Baking,BAKING,test brand @1601644365844,NaN,TEST BRAND @1601644365844,5f77274dbe37ce6b592e90c0,Cogs,5f77274dbe37ce6b592e90bf,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153789,511111706328,Breakfast & Cereal,BREAKFAST_AND_CEREAL,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,5dc1fca91dda2c0ad7da64ae,Cogs,53e10d6368abd3c7065097cc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153790,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,5f494c6e04db711dd8fe87e7,Cogs,5332fa12e4b03c9a25efd1e7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153791,511111400608,Grocery,GROCERY,LIPTON TEA Leaves,0.0,LIPTON TEA LEAVES,5a021611e4b00efe02b02a57,Cogs,5332f5f6e4b03c9a25efd0b4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df[df['brandCode'].isnull()]

,barcode_x,category,categoryCode,name,topBrand,brandCode,id,cpg_type,cpg_id,barcode_y,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
216,511111802075,Baking,BAKING,Kraft Caramels,0.0,NaN,57ebc11fe4b0ac389136a33a,Cogs,559c2234e4b06aca36af13c6,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,511111802075,Baking,BAKING,Kraft Caramels,0.0,NaN,57ebc11fe4b0ac389136a33a,Cogs,559c2234e4b06aca36af13c6,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,511111802075,Baking,BAKING,Kraft Caramels,0.0,NaN,57ebc11fe4b0ac389136a33a,Cogs,559c2234e4b06aca36af13c6,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,511111802075,Baking,BAKING,Kraft Caramels,0.0,NaN,57ebc11fe4b0ac389136a33a,Cogs,559c2234e4b06aca36af13c6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,511111802075,Baking,BAKING,Kraft Caramels,0.0,NaN,57ebc11fe4b0ac389136a33a,Cogs,559c2234e4b06aca36af13c6,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153781,511111501619,Beverages,BEVERAGES,Pepsi Max,0.0,NaN,585a96cbe4b03e62d1ce0e88,Cogs,5332f5fbe4b03c9a25efd0ba,B07BRRLSVC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
153782,511111501619,Beverages,BEVERAGES,Pepsi Max,0.0,NaN,585a96cbe4b03e62d1ce0e88,Cogs,5332f5fbe4b03c9a25efd0ba,B076FJ92M4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
153783,511111501619,Beverages,BEVERAGES,Pepsi Max,0.0,NaN,585a96cbe4b03e62d1ce0e88,Cogs,5332f5fbe4b03c9a25efd0ba,B07BRRLSVC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
153784,511111501619,Beverages,BEVERAGES,Pepsi Max,0.0,NaN,585a96cbe4b03e62d1ce0e88,Cogs,5332f5fbe4b03c9a25efd0ba,B076FJ92M4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


There is lot of nulls in the 'brandCode' column. This will hinder queries when joining these 2 datasets are needed.

In [38]:
 '''
 Now let'see if there are any user ids that are present in receipts but not in users
 '''
 receipts[~receipts['userId'].isin(users['id'])]['userId']

13     5f9c74f7c88c1415cbddb839
15     5ff1e1e9b6a9d73a3a9f10f6
16     5ff1e1dfcfcf6c399c274ab3
20     5f9c74e3f1937815bd2c1d73
21     5ff1e196cfcf6c399c274a38
                 ...           
955    60253861efa6017a44dc6b50
956    60253891b54593795bf69242
966    60253891b54593795bf69242
985    60268c7bb545931ac63683af
990    60268c78efa6011bb151077d
Name: userId, Length: 148, dtype: object

Turns out, there are 148 user ids that's there in receipts data but not in users. This shouldn't happen.